## Weak 36

In [ ]:
import tiktoken
from tokenizers import Tokenizer
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
)
from datasets import load_dataset
import polars as pl
import os

In [ ]:
# Get huggingface dataset
dataset = load_dataset("coastalcph/tydi_xor_rc")
df_train = pl.from_pandas(dataset["train"].to_pandas())
df_test = pl.from_pandas(dataset["test"].to_pandas())

In [ ]:
df_train.columns

In [ ]:
df_train.head()

In [ ]:
df_ar = df_train.filter(pl.col("lang") == "ar")
df_ko = df_train.filter(pl.col("lang") == "ko")
df_te = df_train.filter(pl.col("lang") == "te")

In [ ]:
df_te.describe()

### Get tokenizers and look at sample (Arabic) sentence

In [ ]:
# Load multilingual BERT tokenizer
mbert_tokenizer = Tokenizer.from_pretrained("bert-base-multilingual-uncased")
# Load GPT-4 tokenizer
gpt4_tokenizer = tiktoken.get_encoding("cl100k_base")
# Load NLLB-200 tokenizer
nllb_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
" | ".join(mbert_tokenizer.encode(df_ar["question"][0]).tokens)

In [ ]:
# Decode each token from GPT-4 tokenizer
" | ".join([gpt4_tokenizer.decode([token]) for token in gpt4_tokenizer.encode(df_ar["question"][0])])


In [ ]:
" | ".join(nllb_tokenizer.tokenize(df_ar["question"][0]))

## Get the top 5 most frequent words

In [ ]:
# Language dict
lang_dict = {
    "ar": "arb_Arab",
    "ko": "kor_Hang",
    "te": "tel_Telu",
}

In [ ]:
# Set up translation pipeline for NLLB-200
translator_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
random_txt = df_ko["question"][0]
translator = translator = pipeline(
        "translation",
        model=translator_model,
        tokenizer=nllb_tokenizer,
        src_lang="kor_Hang",
        tgt_lang="eng_Latn",
    )
translator(random_txt)[0]["translation_text"] 

In [ ]:
# Tokenize questions in Arabic using multilingual BERT tokenizer
def _make_translator(src_lang:str):
    translator = pipeline(
        "translation",
        model=translator_model,
        tokenizer=nllb_tokenizer,
        src_lang=src_lang,
        tgt_lang="eng_Latn",
    )
    return translator

def tokenize_question(df: pl.DataFrame) -> pl.DataFrame:
    
    src_lang = lang_dict[df["lang"][0]]
    print(f"Tokenizing and translating {src_lang}...")
    print(f"tydi_xor_rc_{src_lang}.parquet" in os.listdir())
    if f"tydi_xor_rc_{src_lang}.parquet" in os.listdir():
        return pl.read_parquet(f"tydi_xor_rc_{src_lang}.parquet")

    df = df.with_columns(
        pl.col("question")
        .map_elements(lambda x: nllb_tokenizer.tokenize(x), return_dtype=pl.List(pl.Utf8))
        .alias("tokens")
    )

    translator = _make_translator(src_lang)
    df = df.with_columns(
        pl.col("question")
        .map_elements(lambda x: translator(x)[0]["translation_text"])
        .alias("translation")
    )

    df.write_parquet(f"tydi_xor_rc_{src_lang}.parquet")
    df.write_excel(f"tydi_xor_rc_{src_lang}.xlsx")

    return df

In [ ]:
#for df in [df_ar, df_ko, df_te]:
#    df = tokenize_question(df, with_translation=True)
#    print(df.head()["translation"])

df_ar = tokenize_question(df_ar)
df_ar.head()

In [ ]:
count_dict_ar = {}
for tokens in df_ar["tokens"]:
    for token in tokens:
        if token in count_dict_ar:
            count_dict_ar[token] += 1
        else:
            count_dict_ar[token] = 1
# Get as tuple and sort by frequency
sorted_frequency_list_ar = sorted(count_dict_ar.items(), key=lambda x: x[1], reverse=True)

In [ ]:
sorted_frequency_list_ar